# LangChain: Memory


**거대언어모델**과 상호 작용할 때 당연히 **모델은 이전에 말한 내용이나 이전 대화를 기억하지 못합니다**. 이는 Chatbot과 같은 일부 애플리케이션을 구축하고 그들과 대화하고 싶을 때 문제가 됩니다. 따라서 이 섹션에서는 기본적으로 대화의 이전 부분을 기억하고 이를 언어 모델에 공급하여 상호 작용할 때, 대화 흐름을 가질 수 있도록 하는 **메모리**에 대해 다룰 것입니다. LangChain은 이러한 메모리를 관리하기 위한 여러 가지 정교한 옵션을 제공합니다. 

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [76]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [78]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [80]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [82]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [84]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2. Is there anything else you would like to know?'

In [86]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew.'

In [88]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Andrew.


In [90]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Andrew."}

In [92]:
memory = ConversationBufferMemory()

In [94]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [96]:
print(memory.buffer)

Human: Hi
AI: What's up


In [98]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [100]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [102]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

사실상 거대 언어 모델 자체로는 상태가 보존되지 않습니다(stateless). 
언어 모델 자체는 지금까지 진행한 대화를 기억하지 않으니까요. 
**각 트랜젝션과 API 호출은 독립적**이기 때문입니다. 
챗봇이 **메모리를 가지고 있는 것처럼 보이는 이유**는 보통 **코드로 지금까지의 대화 내용 전부를 아주 빠르게 LLM에 맥락으로 제공하기 때문**입니다.
덕분에 메모리는 명시적으로 지금까지 나온 용어와 발언들을 저장할 수 있죠. 
"안녕하세요, 제 이름은 Andrew 입니다. 만나서 반갑습니다." 등등. 
이 저장된 메모리는 LLM에 새로운 입력값이나 추가적인 맥락으로 사용되어 출력값을 생성할 수 있도록 해주죠. 다음 대화 차례 때 마치 이전 대화 내용을 잘 안다는 듯이 말이죠. 그리고 **대화가 길어질수록**, 필요한 메모리의 양이 훨씬 더 늘어나고,
LLM에 많은 토큰을 보내는 **비용도 크게 증가**하게 됩니다.
LLM은 일반적으로 처리하는 토큰 수에 비례하여 요금을 부과하므로, 비용은 더욱 더 비싸질 것입니다.
그래서 LangChain은 여러 가지 대화를 간편하게 저장하고 축적할 수 있는 메모리 종류를 제공합니다. 
지금까지 우리는 **ConversationBufferMemory**를 살펴보았고,
다른 종류의 메모리도 한번 살펴봅시다.

<img src="./memory.png" width="450">

## ConversationBufferWindowMemory

In [106]:
from langchain.memory import ConversationBufferWindowMemory

In [108]:
memory = ConversationBufferWindowMemory(k=1)               

In [110]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [112]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [114]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [115]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [117]:
conversation.predict(input="What is 1+1?")

'1+1 equals 2. Is there anything else you would like to know?'

In [119]:
conversation.predict(input="What is my name?")

"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to know?"

## ConversationTokenBufferMemory

In [123]:
#!pip install tiktoken
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [124]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [127]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [130]:
from langchain.memory import ConversationSummaryBufferMemory

In [132]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [133]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool.'}

In [134]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [138]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool.
Human: What would be a good demo to show?
AI:

> Finished chain.


'For the meeting with the product team, a demo showcasing the latest features and updates of LangChain would be ideal. This could include a live demonstration of how LangChain streamlines language translation processes, improves accuracy, and increases efficiency. Additionally, highlighting any recent success stories or case studies involving LangChain could help showcase its effectiveness to the team.'

In [139]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool.\nHuman: What would be a good demo to show?\nAI: For the meeting with the product team, a demo showcasing the latest features and updates of LangChain would be ideal. This could include a live demonstration of how LangChain streamlines language translation processes, improves accuracy, and increases efficiency. Additionally, highlighting any recent success stories or case studies involving LangChain could help showcase its effectiveness to the team.'}

### Memory types

**ConversationBufferMemory**
* 이 메모리를 사용하면 메시지를 저장할 수 있으며, 변수에서 메시지를 추출할 수 있습니다.

**ConversationBufferWindowMemory**
* 이 메모리는 해당 시간 동안의 대화 상호 작용 목록을 유지합니다. 마지막 K개 상호작용만 사용합니다.

**ConversationTokenBufferMemory**
* 이 메모리는 메모리의 최근 상호 작용 버퍼를 유지하고, 상호 작용 수 대신 토큰 길이를 사용하여 상호 작용 플러시 시기를 결정합니다.

**ConversationSummaryBufferMemory**
* 이 기억은 시간이 지남에 따라 대화의 요약을 생성합니다.


### Additional Memory types

**Vector data memory**
* 대화 또는 다른 곳의 텍스트를 벡터 데이터베이스에 저장하고 가장 관련성이 높은 텍스트 블록을 검색합니다.

**Entity memories**
* LLM을 사용하여, 특정 엔터티에 대한 세부 정보를 기억합니다.

동시에 여러 개의 메모리를 사용할 수도 있습니다.  E.g. Conversation memory + Entity memory to recall individuals.

대화 내용을 기존 데이터베이스(예: SQL의 키-값 저장소)에 저장할 수도 있습니다.